In [1]:
import matplotlib.pyplot as plt
import numpy as np

In [2]:
X = np.array([ [   1,    1,  500,    1],
               [   1,    1,  700,    1],
               [   1,    2,  750,    2],
               [   1,    5,  600,    1],
               [   1,    3, 1450,    2],
               [   1,    0,  800,    1],
               [   1,    5, 1500,    3],
               [   1,   10, 2000,    3],
               [   1,    1,  450,    1],
               [   1,    2, 1000,    2]], dtype = np.float64)

y = np.array([0, 0, 1, 0, 1, 0, 1, 0, 1, 1], dtype = np.float64)

In [3]:
def standard_scale(x):
    res = (x - x.mean()) / x.std()
    return res

In [4]:
X_st = X.copy()
X_st[:, 2] = standard_scale(X[:, 2])
X_st

array([[ 1.        ,  1.        , -0.97958969,  1.        ],
       [ 1.        ,  1.        , -0.56713087,  1.        ],
       [ 1.        ,  2.        , -0.46401617,  2.        ],
       [ 1.        ,  5.        , -0.77336028,  1.        ],
       [ 1.        ,  3.        ,  0.97958969,  2.        ],
       [ 1.        ,  0.        , -0.36090146,  1.        ],
       [ 1.        ,  5.        ,  1.08270439,  3.        ],
       [ 1.        , 10.        ,  2.11385144,  3.        ],
       [ 1.        ,  1.        , -1.08270439,  1.        ],
       [ 1.        ,  2.        ,  0.05155735,  2.        ]])

In [5]:
def sigmoid(z):
    res = 1 / (1 + np.exp(-z))
    return res

**Задание 1.
Измените функцию calc_logloss так, чтобы нули по возможности не попадали в np.log.**

In [6]:
def calc_logloss(y, y_pred):
    y_pred_mod = [x + 0.0001 if x==0 else x for x in y_pred]
    y_pred = np.array([x - 0.0001 if x==1 else x for x in y_pred_mod])
    err = - np.mean(y * np.log(y_pred) + (1.0 - y) * np.log(1.0 - y_pred))
    return err

In [7]:
y1 = np.array([1, 0])
y_pred1 = np.array([1, 0.1])
calc_logloss(y1, y_pred1)

0.05273026032907981

**Задание 2.
Подберите аргументы функции eval_model для логистической регрессии таким образом, чтобы log loss был минимальным.**

In [8]:
def eval_model(X, y, iterations, eta=1e-4):
    np.random.seed(42)
    W = np.random.randn(X.shape[1])
    n = X.shape[0]
    
    for i in range(iterations):
        z = np.dot(X, W)
        y_pred = sigmoid(z)
        err = calc_logloss(y, y_pred)
        
        dQ = 1/n * X.T @ (y_pred - y)
        W -= eta * dQ
        if i % 100 == 0:
            print(i, W, err)
    return W, err

In [9]:
def select_params(etas, n_iters):
    errors_full = {}
    for i in n_iters:
        errors_full[i]=[]

    best_error = np.inf
    best_params = {}

    np.random.seed(42)
    W_start = np.random.randn(X_st.shape[1])
    n = X_st.shape[0]

    for eta in etas:
        for n_iter in n_iters:
            W = W_start.copy()
            curr_eta = eta

            for i in range(n_iter):
                z = np.dot(X_st, W)
                y_pred = sigmoid(z)
                err = calc_logloss(y, y_pred)

                dQ = 1/n * X_st.T @ (y_pred - y)
                W -= curr_eta * dQ
            errors_full[n_iter].append(err)
            #print(curr_eta, n_iter, err, W)
            if err < best_error:
                best_error = err
                best_params = {
                    'eta': curr_eta,
                    'n_iter': n_iter
                }
    #print(f'Лучшая ошибка {best_error} с параметрами {best_params}')
    return best_error, best_params, errors_full

In [10]:
# Сначала рассмотрим большие etas
etas_1 = np.linspace(1, 20, 100)
n_iters_1 = [10, 100, 250, 500, 800, 1000, 5000]

result_1 = select_params(etas_1, n_iters_1)
print(f'Лучшая ошибка {result_1[0]} с параметрами {result_1[1]}')

# Теперь рассмотрим etas<=1
etas_2 = np.linspace(1e-6, 1, 100)
n_iters_2 = [10, 100, 250, 500, 800, 1000, 5000]
result_2 = select_params(etas_2, n_iters_2)
print(f'Лучшая ошибка {result_2[0]} с параметрами {result_2[1]}')

Лучшая ошибка 0.018969185710984854 с параметрами {'eta': 19.808080808080806, 'n_iter': 5000}
Лучшая ошибка 0.1438003796511719 с параметрами {'eta': 1.0, 'n_iter': 5000}


In [11]:
W_1 = eval_model(X_st, y, iterations=800, eta=3)
W1 = W_1[0]
W_2 = eval_model(X_st, y, iterations=800, eta=0.01)
W2 = W_2[0]
print('______________________________________')
print(f'eta={3} W={W_1[0]} LogLoss={W_1[1]}\n')
print(f'eta={0.01} W={W_2[0]} LogLoss={W_2[1]}')

0 [-0.64144649 -4.49089227  0.56554728 -0.17540895] 1.1785958344356262
100 [-8.22246403 -5.69918923  1.37871718 11.50108478] 0.8813807511578317
200 [-13.65808356  -2.03435024  -2.68080052  12.22231623] 0.21824770183199083
300 [-16.35536529  -2.34823417  -3.94277613  13.79514445] 0.19855199112601468
400 [-17.64897713  -2.58113172  -4.3428572   14.8996011 ] 0.1989538691508405
500 [-18.83846034  -2.48013174  -4.70968291  15.83952151] 0.18133358098552604
600 [-19.91794603  -2.49697621  -5.05434406  16.67364291] 0.17253852741296652
700 [-20.90628618  -2.51934508  -5.36365373  17.44307522] 0.16537730858420022
0 [ 0.49292028 -0.15277306  0.64741473  1.51736839] 1.1785958344356262
100 [ 0.29954562 -0.70184086  0.66393101  1.28809857] 0.5200176817937363
200 [ 0.21783205 -0.71711535  0.72830416  1.28039917] 0.5083938298746018
300 [ 0.15081179 -0.71601562  0.78172777  1.29014996] 0.500908097886405
400 [ 0.09214433 -0.71718068  0.82443144  1.306414  ] 0.49535059743739646
500 [ 0.03982235 -0.720647

**Задание 3.
Создайте функцию calc_pred_proba, возвращающую предсказанную вероятность класса 1 
(на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred_proba).**

In [12]:
def calc_pred_proba(X, W):
    y_pred_proba = sigmoid(np.dot(W, X))
    return y_pred_proba

In [13]:
print(f'y_orig = {y}\n')
print(f'W1 = {W1}')
y_pred_proba_1 = calc_pred_proba(X_st.T, W1)
print(f'y_pred_proba_1 = {y_pred_proba_1}')
print(f'W2 = {W2}')
y_pred_proba_2 = calc_pred_proba(X_st.T, W2)
print(f'y_pred_proba_2 = {y_pred_proba_2}')

y_orig = [0. 0. 1. 0. 1. 0. 1. 0. 1. 1.]

W1 = [-21.74954945  -2.34011382  -5.62911512  18.25484958]
y_pred_proba_1 = [4.20531831e-01 6.64609202e-02 9.99996924e-01 1.95613581e-05
 9.02523652e-01 1.87986932e-01 9.99999754e-01 9.23517765e-02
 5.64600397e-01 9.99943977e-01]
W2 = [-0.09161818 -0.73872879  0.9256099   1.40510823]
y_pred_proba_2 = [0.41776462 0.51245225 0.69245945 0.04327322 0.80362866 0.72699631
 0.80724768 0.21299272 0.39474721 0.78395463]


**Задание 4.
Создайте функцию calc_pred, возвращающую предсказанный класс 
(на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred).**

In [14]:
def calc_pred (X, W):
    y_pred = sigmoid(np.dot(W, X))
    y_pred[y_pred <= 0.5] = 0
    y_pred[y_pred > 0.5] = 1
    return y_pred

In [15]:
print(f'y_orig = {y}\n')

print(f'W1 = {W1}')
y_pred_1 = calc_pred(X_st.T, W1)
print(f'y_pred_1 = {y_pred_1}')
print(f'W2 = {W2}')
y_pred_2 = calc_pred(X_st.T, W2)
print(f'y_pred_2 = {y_pred_2}')

y_orig = [0. 0. 1. 0. 1. 0. 1. 0. 1. 1.]

W1 = [-21.74954945  -2.34011382  -5.62911512  18.25484958]
y_pred_1 = [0. 0. 1. 0. 1. 0. 1. 0. 1. 1.]
W2 = [-0.09161818 -0.73872879  0.9256099   1.40510823]
y_pred_2 = [0. 1. 1. 0. 1. 1. 1. 0. 0. 1.]
